In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Ridge plot
Ridge plots show each trace from your calcium session offset a little bit on the y-axis. They can be a useful way to get an initial big-picture view of your data. In this notebook, we will recreate the following plot: 

![Example](data/ridge_example.png)

## Load data and build cnmf object
Start cluster and build object from `hdf5` file saved from data already analyzed in the CNMFE demo in caiman:

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import caiman as cm
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.source_extraction import cnmf

In [ ]:
cnmfe_results_path = r'./data/cnmfe_demo.hdf5'
num_cpus = 2 # use however many you need: just leave a few so your RAM stays happy
# note if a cluster already exists it will be closed so a new session will be opened
if 'dview' in locals():  # locals contains list of current local variables
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', 
                                                 n_processes=num_cpus, 
                                                 single_thread=False,
                                                 ignore_preexisting=True)
#Number of nodes in cluster 
print(f"Cluster has {n_processes} processes in the pool");
cnm = load_CNMF(cnmfe_results_path, 
                n_processes, 
                dview=dview)

## Extract basic info from estimates object
Get denoised traces from cnmf object. Also, reconstruct array of frame times. 

Note we will focus on the "good" components in `cnm.estimates.idx_components` (for more on interpreting the properties in `cnm.estimates`, see https://caiman.readthedocs.io/en/master/Getting_Started.html#result-interpretation).

In [ ]:
good_inds = cnm.estimates.idx_components
denoised_traces = cnm.estimates.C[good_inds,:]  #num_components x num_frames

num_components = denoised_traces.shape[0]
num_samples = denoised_traces.shape[1]
frame_rate = cnm.params.data['fr']
sampling_pd = 1/frame_rate
num_frames = num_samples
frame_times = np.arange(0, sampling_pd*num_samples, sampling_pd)

## Define ridge plot
This is a pretty straightforward function: just plot each trace, slightly offset from its neighbor. As discussed below, the main wrinkles have to do with how to handle the spacing of the traces and the ytick labels:

In [ ]:
def ridge_plot(array, 
               times,
               trace_spacing=5,
               ytick_spacing=10,
               title=None,
               color='black',
               alpha=0.5,
               line_width=1, 
               ax = None):
    """
    Plot ridge plot of all rows in array, given x values.
    Inputs:
        array: num_components x num_times array of traces
        times: 1-d array 
        trace_spacing (scalar): distance between each plot on y axis
        ytick_spacing (int): period between ytick labels (every ytick_spacing traces)
        color: line color (r,g,b) or standard matplotlib color string
        alpha (scalar): alpha for each trace, if you want them more see-through when density is high.
        line_width (scalar): how wide?
        ax: axes object if you want to draw in pre-existing axes (None if you want new axes)
    
    Outputs:
        ax: axes object with lines drawn
    """
    num_traces = array.shape[0]
    num_yticks = int(1+num_traces//ytick_spacing)
    
    # set y position of each trace
    y_position_traces = np.linspace(0, num_traces*trace_spacing, num=num_traces) 
    
    # set y tick properties 
    y_ticks = np.linspace(0, (num_traces-1)*trace_spacing, num=num_yticks)
    y_tick_labels = np.arange(0, num_traces+2*ytick_spacing, ytick_spacing, dtype=np.uint8) # +2*y_tick_spacing just for insurance
    y_tick_labels = y_tick_labels[:num_yticks]
    
    if ax is None:
        f, ax = plt.subplots()
    for ind, line in enumerate(array):
        ax.plot(times, 
                line+y_position_traces[ind], 
                color=color,
                alpha=alpha,
                linewidth=line_width)
    # only show left/bottom axis lines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ylims = ax.get_ylim()
    # set ylimits to make it pretty (this could use some tweaking probably)
    ax.set_ylim(0.1*ylims[0], ylims[1]-0.05*ylims[1])
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_tick_labels)
    plt.autoscale(enable=True, axis='x', tight=True)
    return ax
print('done')

## Using ridge_plot()
As mentioned above, there are two spacing parameters that are the only real wrinkles here: `trace_spacing` and `ytick_spacing`.  

The `ytick_spacing` parameter sets how frequently to display ytick labels (labeling every 10th trace is the default). The `trace_spacing` parameter determines how far apart traces are from each other on the y axis. The best value will depend on the scale of your traces.  You will likely want to get a sense of the maximum values in your array and scale it based on that, as I do in the following example:

In [ ]:
component_spacing = int(np.max(denoised_traces)*0.1)
print(f"Traces will be spread {component_spacing} units apart")

In [ ]:
f, traces_ax = plt.subplots(figsize=(19,10))
traces_ax = ridge_plot(denoised_traces, 
                       frame_times, 
                       color='black',
                       trace_spacing=component_spacing, 
                       ytick_spacing=5,
                       alpha=0.6,
                       line_width=0.75,
                       ax=traces_ax)
traces_ax.set_xlabel("Time (s)", fontsize=12)
traces_ax.set_ylabel("Component #", fontsize=12)
traces_ax.set_title("Ridge Plot Example", fontsize=16)
plt.tight_layout()

If this was a plotting library we would likely just wrap that trace spacing calculation into `ridge_plot()`, but for now it's just a parameter you can explore.

## Doing things with your ridge plot
Once you have a ridge plot you like, there are a few simple things you can do. 

For instance, you can add a rectangular highlight (e.g., when a stimulus was presented):

In [ ]:
rect_overlay = traces_ax.axvspan(25, 45, color='yellow', alpha=0.3)

You can remove the rectangle easily if you want to build animations with scrolling rectangles that sweep across the ridge plot:

In [ ]:
rect_overlay.remove()

If you ever want to build an oscilloscope-like animation, you will need an efficient way to remove lines between sweeps. (Note Matplotlib will not be fast enough for real-time display of data, but if you just want to create a high-quality animation for a presentation, it is fine).

The following function will remove all the lines from your ridge plot:

In [ ]:
def remove_lines(ax):
    for line in ax.get_lines(): # ax.lines:
        line.remove()
    return ax

In [ ]:
traces_ax = remove_lines(traces_ax)

It is left as an exercise to write a function to add lines back to the axes object. 